# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules:

In [1]:
import gsd.hoomd
import numpy as np

from morphct.chromophores import amber_dict, get_chromo_ids_smiles
from morphct.system import System

In the cell below, we'll create a system object for morphct. This the main class that will hold all the information for our simulation. We'll need to give it a gsd file, path to an output directory, the frame of the gsd file to use, the scaling factor to convert the lengths in the gsd to Angstroms, and a dictionary to map particle types to elements. Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 2 p3ht 16-mers: 

In [2]:
gsdfile = "p3ht-trajectory.gsd"

system = System(gsdfile, "output_p3ht", frame=-1, scale=3.5636, conversion_dict=amber_dict)
system.visualize_system()

/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/mbuild/utils/io.py:29: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.py

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next let's use SMARTS matching to detect our chromophores. This SMARTS string shown below is for p3ht. 

Note: The positions/orientations in the final frame of the gsd file are not optimal, so we are using the first frame (before any distortion) for SMARTS matching and then mapping those indices to the final structure. Sometimes SMARTS matching is not the best method to find the chromophores--check the ITIC example to see another method.

In [3]:
smarts_str = "c1cscc1CCCCCC"

with gsd.hoomd.open(gsdfile) as f:
    snap0 = f[0]

aaids = get_chromo_ids_smiles(snap0, smarts_str, system.conversion_dict)

Found 32 chromophores.


Next let's add these chromophores to the system and visualize them. We only have donor species in this system, and they'll be colored purple.

In [4]:
# the indices of the donor in one molecule
# saved for use in morphct-flow
np.savetxt("p3ht_d_ids.csv", aaids[:16], fmt="%i")

In [5]:
system.add_chromophores(aaids, "donor")

system.visualize_chromophores()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next let's compute the energies required to run the KMC simulation. First, the neighbors will be calculated (using voronoi analysis) and then the single and dimer energies will be calculated and saved to a file. (So that the simulation can be restarted from this point more easily.)

In [6]:
system.compute_energies()

There are 192 chromophore pairs
Starting singles energy calculation...
Finished in 10.76 s. Output written to output_p3ht/singles_energies.txt.
Starting dimer energy calculation...
Finished in 101.27 s. Output written to output_p3ht/dimer_energies.txt.


We can check that the pair and singles inputs look reasonable. There won't be any bonds and hydrogen atoms should've been added.

In [7]:
i = 86 # try any number from 0 to 191
print(f"Pair #{i}:")
system.visualize_qcc_input(i, single=False)

i = 0 # try any number from 0 to 31
print(f"Single #{i}:")
system.visualize_qcc_input(i)

Pair #86:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Single #0:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Once the energy files are finished, we can use them to set the energy values of the chromophores . (This can also be run to restart the calculation from this point.)

In [8]:
system.set_energies()

Energies set.


This function sets the `homo_1`, `homo`, `lumo`, `lumo_1`, `neighbors_delta_e`, and `neighbors_ti` attributes of each chromphore:

In [9]:
i = 0
chromo = system.chromophores[i]
print(f"Chromophore {i}:")
print(
    f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} "
    f"LUMO+1: {chromo.lumo_1:.2f}"
)
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

Chromophore 0:
HOMO-1: -9.01 HOMO: -8.51 LUMO: 0.55 LUMO+1: 0.60
17 neighbors
DeltaE of first neighbor: 0.079
Transfer integral of first neighbor: 0.594


With all the energy values set, we're ready to run KMC! We need to set the temperature that the KMC simulation will be run at and the lifetimes and numbers of our carriers:

In [10]:
lifetimes = [1e-14, 1e-13, 1e-12, 1e-11]
temp = 300
system.run_kmc(lifetimes, temp, n_holes=10, verbose=1)

All KMC jobs completed!
Combining outputs...
---------- KMC_ANALYZE ----------
All figures saved in output_p3ht/kmc/figures
---------------------------------
Considering the transport of hole...
Obtaining mean squared displacements...
	Notice: The data from 13 carriers were
	discarded due to the carrier lifetime being more than double
	(or less than half of) the specified carrier lifetime.
Plotting distribution of hole displacements


/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/ryn/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/matplotlib/__in

	Figure saved as hole_displacement_dist.png
Calculating mobility...
	Standard Error 2.1850092557378824e-07
	Fitting r_val = 0.2510333504896112
	Figure saved as lin_MSD_hole.png
	Figure saved as semi_log_MSD_hole.png
	Figure saved as log_MSD_hole.png
	----------------------------------------
	Hole mobility = 5.17E-03  +/- 1.91E-03 cm^2 V^-1 s^-1
	----------------------------------------
Calculating hole trajectory anisotropy...
	----------------------------------------
	Hole charge transport anisotropy: 0.518
	----------------------------------------
Plotting hole hop frequency distribution...
	DYNAMIC CUT
	Notice: No minima found in distribution. Cutoff set to None.
	Cluster cut-off based on hop frequency set to None
	Figure saved as total_hop_freq_hole.png
Plotting hole net hop frequency distribution...
	Figure saved as net_hop_freq_hole.png
Plotting (total - net hops) discrepancy distribution...
	There are 7 paths with one-way transport.
	There are 14 paths with total - net < 10.
	Fi

The output files for each process are saved in `output_p3ht/kmc/kmc_PROC#.log` (where `PROC#` is whatever process number the job was run on) and analysis plots will be saved in `output_p3ht/kmc/figures/`.

In [11]:
with open("output_p3ht/kmc/kmc_00.log", "r") as f:
    lines = f.readlines()
print(*lines)

Found 5 jobs to run
 starting job 0
 	hole hopped 0 times over 0.00e+00 seconds into image [0 0 0] for a displacement of
 	0.00 (took walltime 0.00 seconds)
 starting job 1
 	hole hopped 8 times over 9.74e-14 seconds into image [0 0 0] for a displacement of
 	15.22 (took walltime 0.01 seconds)
 starting job 2
 	hole hopped 0 times over 0.00e+00 seconds into image [0 0 0] for a displacement of
 	0.00 (took walltime 0.00 seconds)
 starting job 3
 	hole hopped 243 times over 9.97e-13 seconds into image [0 0 0] for a displacement of
 	13.04 (took walltime 0.32 seconds)
 starting job 4
 	hole hopped 0 times over 0.00e+00 seconds into image [0 0 0] for a displacement of
 	0.00 (took walltime 0.00 seconds)

